<style>
#notebook-container {
    padding: 15px;
    background-color: #fff;
    min-height: 0;
    -webkit-box-shadow: 0px 0px 0px 0px rgba(87, 87, 87, 0.2);
    box-shadow: 0px 0px 0px 0px rgba(87, 87, 87, 0.2);
</style>

vers. 1.0.0  

<a href="../economy/index.html">На главную страницу</a>

In [1]:
from IPython.display import HTML

HTML('''
<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>
''')

# Список крупнейших компаний России
## Количество работников

Некоторое представление об экономики России, можно составить на основе анализа деятельности крупнейших компаний. Такой подход применяется рядом бизнес-изданий: 
- <a href="https://www.rbc.ru/rbc500/"> Рейтинг РБК 500 :Весь бизнес России</a>
- <a href="https://expert.ru/dossier/rating/expert-400/"> Эксперт. Эксперт 400 - рейтинг ведущих российских компаний</a>
- <a href="http://www.forbes.ru/rating/367067-200-krupneyshih-rossiyskih-chastnyh-kompaniy-2018-reyting-forbes"> Форбс. Топ-200 крупнейших российских частных компаний</a>
- <a href="http://www.forbes.ru/rating/369567-zarubezhnyy-kapital-reyting-krupneyshih-inostrannyh-kompaniy-2018"> Форбс. Топ-50 крупнейших иностранных компаний в России</a>
- <a href="https://www.kommersant.ru/apps/117791"> Коммерсант. 600 крупнейших компаний России</a>

Набор данных сформирован перекрестно из нескольких открытых источников. 

В расчете данные компаний АФК "Система", Госкорпораци "Росатом" учитываются совокупно, Госкорпорация "Ростех" учитывается раздельно по ключевым предприятиям. Предприятия Госкорпорации "Роскосмос" учитываются раздельно по ключевым компаниям космической отрасли. Отраслевыя структура соответствует методике РБК.

Описание набора данных:
- Компании в списке упорядочены по величине выручки
- Компании имею порядковый индекс от 0 до n. Первая в списке организация имеет индекс 0, последняя - индекс n. 
- Всего данных о численности персонала за 5 лет с 2013 по 2017 года
- Данные о количестве персонала представлены в столбцах: от <b><samp>p2013</samp></b> - для численности персонала за 2013 год, до <b><samp>p2017</samp></b> - для численности персонала за 2017 год.

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

plt.style.use('ggplot')
pd.options.display.max_rows = 800

df = pd.read_excel('./../data/data.xlsx')
df = df.drop(['city', 'v2013', 'v2014', 'v2015', 'v2016', 'v2017', 
             'i2013', 'i2014', 'i2015', 'i2016', 'i2017', 'v'], axis=1)

In [3]:
v = {'2013': df.p2013.sum().astype(int), 
     '2014': df.p2014.sum().astype(int), 
     '2015': df.p2015.sum().astype(int),
     '2016': df.p2016.sum().astype(int),
     '2017': df.p2017.sum().astype(int)}


comp = {'2013': df.p2013.isnull().value_counts()[0],
        '2014': df.p2014.isnull().value_counts()[0],
        '2015': df.p2015.isnull().value_counts()[0],
        '2016': df.p2016.isnull().value_counts()[0],
        '2017': df.p2017.isnull().value_counts()[0]}

comp_len = {'2013': len(df.p2013),
            '2014': len(df.p2014),
            '2015': len(df.p2015),
            '2016': len(df.p2016),
            '2017': len(df.p2017)}

In [4]:
print('\nХарактеристика набора данных: ')
dfv = pd.DataFrame([v, comp_len, comp], index=['company', 'comp', 'data']).T

dfv = dfv.sort_index(ascending=False)
dfv['nodata'] = dfv.comp - dfv.data
dfv['perc_data'] = (dfv.data * 100 / dfv.comp).round(1)
dfv['perc_nodata'] = (100 - dfv.perc_data).round(1)

dfv = dfv[['comp', 'data', 'nodata', 'perc_data', 'perc_nodata', 'company']]
dfv.columns = ['всего компаний', 'есть данные', 'нет данных', 'есть данные %', 'нет данных %', 'всего работников']
dfv


Характеристика набора данных: 


,всего компаний,есть данные,нет данных,есть данные %,нет данных %,всего работников
2017,661,372,289,56.3,43.7,7829758
2016,661,241,420,36.5,63.5,6755333
2015,661,289,372,43.7,56.3,7480817
2014,661,253,408,38.3,61.7,7082535
2013,661,237,424,35.9,64.1,6693889


### Набор данных

<pre>
Столбцы: 
- industry       - отрасль к оторой относится компания
- p2013 - p2017  - количество работников в компании (человек)
- forg           - иностранная компания
- state          - компания с госучастием, государство основной собственник
- Nan            - нет данных (в ячейке)
</pre>

In [5]:
df = df.round()
df

,name,industry,p2013,p2014,p2015,p2016,p2017,forg,country,state
0,Газпром,Нефть и газ,459500.0,459600.0,462400.0,467400.0,463000.0,0,Россия,1
1,ЛУКОЙЛ,Нефть и газ,109600.0,110300.0,106200.0,105500.0,103600.0,0,Россия,0
2,Роснефть,Нефть и газ,228000.0,248900.0,261500.0,295800.0,302100.0,0,Россия,1
3,Сбербанк России,Финансы,306123.0,329566.0,330677.0,325075.0,310300.0,0,Россия,1
4,Российские железные дороги,Транспорт,902000.0,835000.0,809000.0,774000.0,755000.0,0,Россия,1
5,ВТБ,Финансы,103808.0,101072.0,92882.0,94966.0,96957.0,0,Россия,1
6,X5 Retail Group,Торговля,102100.0,117400.0,147498.0,196128.0,250874.0,0,Россия,0
7,Сургутнефтегаз,Нефть и газ,118000.0,118000.0,115218.0,117000.0,116000.0,0,Россия,0
8,Магнит,Торговля,217258.0,257551.0,265982.0,271369.0,276290.0,0,Россия,0
9,Росатом,Атомная промышленность,255300.0,255000.0,256600.0,249940.0,247300.0,0,Россия,1
